# Snow-cover attributes and time-series extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the snow-cover time-series from the MODIS dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Observations
* This notebook assumes that the GEE code to export snow-cover mean time-series from the MODIS dataset were run before in the GEE platform and that the output CSV-files are locally available. 
* It is not possible to export the 15,000 catchments at one single CSV, so there might be many files with the time-series stored separetly. 

## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/shapefiles/estreams_catchments.shp
* data/gee/snowcover/EStreams_modis_snow_cover_mean_gee_{}.csv. Snow-cover time-series CSV-file(s) exported from GEE

**Directory:**
* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Hall, D. K., V. V. Salomonson, and G. A. Riggs. 2016. MODIS/Terra Snow Cover Daily L3 Global 500m Grid. Version 6. Boulder, Colorado USA: NASA National Snow and Ice Data Center Distributed Active Archive Center.

## Licenses

* Snow Cover: Open access: You may download and use photographs, imagery, or text from the NSIDC web site, unless limitations for its use are specifically stated. For more information on usage and citing NSIDC datasets, please visit the [NSIDC 'Use and Copyright' page] (https://nsidc.org/about/data-use-and-copyright).(Last access 27 November 2023) 

## Observations
* This notebook assumes that the GEE code to export snow cover time-series from the MODIS dataset (EStreams_landscape_timeseries_Snow_Cover_gee.txt) was run before in the GEE platform and that the output CSV-files are locally available. 

# Import modules

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
import glob

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [ ]:
# Non-editable variables:
PATH_OUTPUT_TS = "results/timeseries/snowcover"
PATH_OUTPUT_ST = "results/staticattributes"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [ ]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

## GEE outputs
### Snow-cover

In [ ]:
# Check the files in the subdirectory:
filenames = glob.glob("data/gee/snowcover/*.csv")
print("Number of files:", len(filenames))
print("First file:", filenames[0])

In [ ]:
# First, we create an empty DataFrame for the data with a datetime index:
snowcover_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # Read the data from the CSV file:
    snowcover_file = pd.read_csv(file)
    snowcover_file.drop(["system:index", ".geo"], axis=1, inplace=True)
    snowcover_file = snowcover_file.T
    
    # Set columns based on the "basin_id" row and drop it
    snowcover_file.columns = snowcover_file.loc["basin_id", :].tolist()
    snowcover_file.drop(["basin_id"], axis=0, inplace=True)
    
    # Convert the index to integers and sort it
    snowcover_file.index = snowcover_file.index.astype(int)
    snowcover_file.sort_index(inplace=True)
    
    # Create a new DataFrame with datetime index and assign values
    snowcover_file_df = pd.DataFrame(columns=snowcover_file.columns)
    snowcover_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
    snowcover_file_df.loc[:, snowcover_file.columns] = snowcover_file
    snowcover_file_df.set_index("dates", inplace=True)
    snowcover_file_df.index.name = ""
    
    # Concatenate the DataFrames along the columns (axis=1)
    snowcover_df = pd.concat([snowcover_df, snowcover_file_df], axis=1)
    
snowcover_df

In [ ]:
# Here we add the columns of the catchemnts that were not processed
# Adding new columns with NaN values only if they don't exist
for col in catchment_boundaries.basin_id.tolist():
    if col not in snowcover_df.columns:
        snowcover_df[col] = np.nan
snowcover_df

In [ ]:
# Here we sort the columns:
snowcover_df = snowcover_df.sort_index(axis=1)

snowcover_df

In [ ]:
# Resample to yearly mean
snowcover_yr = snowcover_df.resample('Y').mean()

snowcover_yr

In [ ]:
# Calculate the mean for each month across all years (monht of the year)
snowcover_moy = snowcover_df.groupby(snowcover_df.index.month).mean()

# Rename the index to the three-letter month abbreviations
snowcover_moy.index = pd.to_datetime(snowcover_moy.index, format='%m').strftime('%b')

snowcover_moy

# Final aggregation (static attributes)

In [ ]:
# Snow cover:
snowcover_moy_T = snowcover_moy.T
snowcover_moy_T.columns = pd.to_datetime(snowcover_moy_T.columns, format='%b').strftime('%m')
snowcover_moy_T.columns = "sno_cov_" + snowcover_moy_T.columns
snowcover_moy_T["sno_cov_mean"] = snowcover_moy_T.mean(axis = 1)
snowcover_moy_T

In [ ]:
# Assign the "basin_id" to the gauges names:
snowcover_moy_T.index.name = "basin_id"

In [ ]:
# Assign the "date" as the index name of the df:
snowcover_yr.index.name = "date"
snowcover_df.index.name = "date"

In [ ]:
# Round the data to 3 decimals
snowcover_df = snowcover_df.astype(float).round(3)
snowcover_yr = snowcover_yr.astype(float).round(3)
snowcover_moy_T = snowcover_moy_T.astype(float).round(3)

# Data export

In [ ]:
# Export the final datasets:
# Time-series:
snowcover_df.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_monhtly.csv")
snowcover_yr.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_yearly.csv")

# Static attributes:
snowcover_moy_T.to_csv(PATH_OUTPUT_ST+"/estreams_snowcover_attributes.csv")

# End